In [1]:
from Bio import SeqIO

gene_file = 'hg19_gene_clean.fa'

e = 0
gene_seqs = []
gene_ids = []
for gene in SeqIO.parse('/home/ubuntu/data/' + gene_file, 
                        'fasta'):
    
    cutoff = 200
    if len(str(gene.seq)) < cutoff:
        continue

    gene_ids.append(str(gene.id))
    s_gene = str(gene.seq)[0:cutoff]
    gene_seqs.append(s_gene)

    e = e + 1
    if e%10000 == 0:
        print('Finished ' + str(e) + ' genes')

def getKmers(sequence, size):
    return [sequence[x:x+size].upper() for x in range(len(sequence) - size + 1)]

kmer = 10
gene_texts = [' '.join(getKmers(i, kmer)) for i in gene_seqs]

Finished 10000 genes
Finished 20000 genes
Finished 30000 genes


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings('ignore')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(gene_texts)

encoded_docs = tokenizer.texts_to_sequences(gene_texts)
max_length = max([len(s.split()) for s in gene_texts])
X_gene = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

print(X_gene)
print('\n')
print(X_gene.shape)

Using TensorFlow backend.


[[547859 642875 642876 ... 125875 110809  77277]
 [253351 294309 164774 ...  31487 125876 218639]
 [164781  61751 110810 ... 253371 218650 294330]
 ...
 [300782 349547 453994 ...  14457  10644  10929]
 [ 63780 142785 252925 ...  53054 543738 324349]
 [273128 360570 109937 ... 219163  82309 120347]]


(35549, 191)


In [3]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

891013


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
from tensorflow.keras.layers import Conv1D, LSTM, Dense, MaxPooling1D, Flatten, Dropout, Embedding, Activation, Bidirectional

model = Sequential()
model.add(Embedding(944238, 10))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.load_weights("LSTM.weights.best.hdf5")
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
model.summary()

In [ ]:
gene_predictions = model.predict_classes(X_gene)
gene_predictions_prob = model.predict_proba(X_gene)

In [ ]:
X_gene.shape

In [ ]:
gene_predictions.shape

In [ ]:
gene_predictions_prob.shape

In [ ]:
import pandas as pd
gene_pred_df = pd.DataFrame({'Gene': gene_ids, 
                             'Predict': list(gene_predictions.flatten()), 
                             'Prob': list(gene_predictions_prob.flatten())})
gene_pred_df = gene_pred_df.sort_values(['Prob'], ascending = False)
gene_pred_df[(gene_pred_df['Predict'] == 1) & (gene_pred_df['Prob'] > 0.8)]